In [1]:
from services.notebook.model_loader import ModelLoader
import services.adapters.sqlite_adapter as sa
from services.discovery.dataset_discovery import EventDatasetDiscovery
from tests.samples.sources import SIMPLE_BIG_DATA
from datetime import datetime
import services.notebook.visualization as vis

adapter = sa.SQLiteAdapter(SIMPLE_BIG_DATA)
discovery = EventDatasetDiscovery(
    SIMPLE_BIG_DATA, adapter, datetime(2021, 1, 1), datetime(2022, 1, 1)
)
dd = discovery.discover_dataset()
ml = ModelLoader()
m = ml.create_dataset_model(dd)
m._to_globals(globals())

services.notebook.model_loader._dataset_model

In [2]:
app_install

Empty DataFrame
Columns: []
Index: []


KeyError: 'unique_user_count'